## Use taxonomic read classifications from MEGAN6 to extract day and treatment Phylum-specific FastQs from Arthropoda and Alveolata

In [1]:
%%bash
echo "TODAY'S DATE:"
date
echo "------------"
echo ""
#Display operating system info
lsb_release -a
echo ""
echo "------------"
echo "HOSTNAME: "; hostname 
echo ""
echo "------------"
echo "Computer Specs:"
echo ""
lscpu
echo ""
echo "------------"
echo ""
echo "Memory Specs"
echo ""
free -mh

TODAY'S DATE:
Fri Jan 31 08:14:52 PST 2020
------------

Distributor ID:	Ubuntu
Description:	Ubuntu 16.04.6 LTS
Release:	16.04
Codename:	xenial

------------
HOSTNAME: 
swoose

------------
Computer Specs:

Architecture:          x86_64
CPU op-mode(s):        32-bit, 64-bit
Byte Order:            Little Endian
CPU(s):                24
On-line CPU(s) list:   0-23
Thread(s) per core:    2
Core(s) per socket:    6
Socket(s):             2
NUMA node(s):          1
Vendor ID:             GenuineIntel
CPU family:            6
Model:                 44
Model name:            Intel(R) Xeon(R) CPU           X5670  @ 2.93GHz
Stepping:              2
CPU MHz:               2925.815
BogoMIPS:              5851.59
Virtualization:        VT-x
L1d cache:             32K
L1i cache:             32K
L2 cache:              256K
L3 cache:              12288K
NUMA node0 CPU(s):     0-23
Flags:                 fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush dts acpi mmx fxsr

No LSB modules are available.


### Set variables

In [2]:
# Set data directories
%env crab_data=/home/sam/data/C_bairdi/RNAseq
%env hemat_data=/home/sam/data/Hematodinium/RNAseq
%env wd=/home/sam/analyses

# Programs
%env seqtk=/home/sam/programs/seqtk-1.3/seqtk

env: crab_data=/home/sam/data/C_bairdi/RNAseq
env: hemat_data=/home/sam/data/Hematodinium/RNAseq
env: wd=/home/sam/analyses
env: seqtk=/home/sam/programs/seqtk-1.3/seqtk


#### Input data are here:

FastAs: https://gannet.fish.washington.edu/Atumefaciens/20200114_cbai_MEGAN_read_extractions/

Trimmed-FastQs: https://gannet.fish.washington.edu/Atumefaciens/20191218_cbai_fastp_RNAseq_trimming/

### Extract phylum-specific reads from trimmed FastQ files


Use FastA IDs from MEGAN6 taxonomic read extraction FastAs to pull out appropriate reads from each phylum (Arthropoda and Alveolata). This is performed because MEGAN6 strips paired read ID after the first space. As such, the resulting read extractions using MEGAN end up with a FastA file containing two reads with identicial headers. Not sure if this will cause any downstream issues (i.e. with Trinity) where paired end data is used, so playing it safe and using the truncated IDs to pull FastQs with complete sequence headers for use in subsequent data wrangling.

In [5]:
%%bash

timestamp=$(date +%Y%m%d)

# Create associative arrays

## Infection status
declare -A inf_status_array=( [329774]=infected [329775]=uninfected [329776]=infected [329777]=uninfected )
## Sampling day
declare -A sample_day_array=( [329774]=D12 [329775]=D12 [329776]=D26 [329777]=D26 )


for directory in ${crab_data} ${hemat_data}
do
	# Get species name
	species=$(echo "${directory}" | awk -F"/" '{print $5}')
    
    # Make new directory and change to that directory ("$_" means use previous command's argument)
    mkdir --parents "${wd}"/"${timestamp}"."${species}"_megan_reads \
    && cd "$_" || exit

	# Set seqtk list filename
	seqtk_list=${timestamp}.${species}.seqtk.read_id.list

	# Set output FastQ filenames
    prefix=${timestamp}.${species}
    R1_suffix=megan_R1.fq
    R2_suffix=megan_R2.fq

	######################################################
	# Create FastA IDs list to use for sequence extraction
	######################################################
	for fasta in "${directory}"/*.fasta
	do
      echo "Pulling FastA IDs from ${fasta}"
      echo ""
      grep ">" "${fasta}" | awk 'sub(/^>/, "")'
	done | sort -u >> "${seqtk_list}"
    
    
    echo ""
    echo "Finished with FastA ID extraction."
    echo ""
    echo "Moving on to read extractions..." 
    echo ""
    echo ""
    
    
    ######################################################
	# Extract corresponding R1 and R2 reads using seqtk FastA ID list
    ######################################################
	for fastq in "${directory}"/*R1*.gz
	do
      # Strip path from filename
      fastq_nopath=${fastq##*/}
                     
      # Get sample ID from FastQ filename
      sample=$(echo "${fastq_nopath}" | awk -F "_" '{print $1}')
    
      # Pull infection status and sample day from associative array
      inf_status=${inf_status_array[$sample]}
      sample_day=${sample_day_array[$sample]}
        
      # Set output filename  
      R1_out="${prefix}.${sample_day}.${inf_status}.${R1_suffix}"
      
 
      echo "Extracting R1 reads from ${fastq}."
      echo ""
      echo "Writing R1 reads to ${R1_out}"
    
      # Use seqtk to pull out desired FastQ reads
	  ${seqtk} subseq "${fastq}" "${seqtk_list}" >> "${R1_out}"
	done
    
    echo ""
    echo "Done with R1 read extractions"
    echo ""
    echo "------------"
    echo ""

	for fastq in "${directory}"/*R2*.gz
	do
       # Strip path from filename
      fastq_nopath=${fastq##*/}
                     
      # Get sample ID from FastQ filename
      sample=$(echo "${fastq_nopath}" | awk -F "_" '{print $1}')
                     
      # Pull infection status and sample day from associative array  
      inf_status=${inf_status_array[$sample]}
      sample_day=${sample_day_array[$sample]}
        
      # Set output filename    
      R2_out="${prefix}.${sample_day}.${inf_status}.${R2_suffix}"
      
 
      echo "Extracting R2 reads from ${fastq}."
      echo ""
      echo "Writing R2 reads to ${R2_out}"
	  ${seqtk} subseq "${fastq}" "${seqtk_list}" >> "${R2_out}"
	done
    
    echo "-------------------------------------"
    echo ""
    # Print working directory and list files
    pwd
	ls -ltrh
    echo ""
    echo "-------------------------------------"
    echo ""
done


Process is terminated.
